# Model 1: Train and Evaluation Fully Connected Neural Network

In [ ]:
from pathlib import Path
import torch 
import numpy as np
from torch.utils.data import DataLoader

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
import sys; sys.path.insert(0, '/mnt/src')

## Read Datasets from .csv

In [ ]:
from utils.file_io import read_angle_datasets

In [ ]:
feature_columns = [
    'left_boom_base_yaw_joint', 
    'left_boom_base_pitch_joint',
    'left_boom_main_prismatic_joint',
    'left_boom_second_roll_joint',
    'left_boom_second_yaw_joint',
    'left_boom_top_pitch_joint',
    'left_boom_ee_joint',
    'cable1_property(length,youngsmodule(bend,twist))',
    'cable2_property(length,youngsmodule(bend,twist))',
    'cable3_property(length,youngsmodule(bend,twist))'
]

label_features = [
    ('cable1_lowest_point', np.array([2], dtype=np.int64)),
    ('cable2_lowest_point', np.array([2], dtype=np.int64)),
    ('cable3_lowest_point', np.array([2], dtype=np.int64))
]

normalized_features = [
    ('cable1_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64)),
    ('cable2_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64)),
    ('cable3_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64))
]

In [ ]:
data_folder = Path("/mnt/data/")
train_data, test_data = read_angle_datasets(data_folder, 0.9, feature_columns, label_features, normalized_features=normalized_features)

In [ ]:
input_shape, output_shape = train_data[0][0].shape[0], train_data[0][1].shape[0]
print(f"Data shape {input_shape} / {output_shape} of total {len(train_data) + len(test_data)} data rows!")

## Load parameter, functions and dataloader

In [ ]:
from utils.file_io import define_dataloader_from_angle_dataset
import os
import ast
from dotenv import load_dotenv

In [ ]:
model_path = Path("/mnt/models/fully_connected/").absolute()

In [ ]:
dotenv_path = model_path / ".env"
load_dotenv(dotenv_path=dotenv_path)

learning_rate = float(os.getenv("LEARNING_RATE"))
batch_size = int(os.getenv("BATCH_SIZE"))
hidden_activation = os.getenv("HIDDEN_ACTIVATION")
optimizer = os.getenv("OPTIMIZER")
hidden_layers = ast.literal_eval(os.getenv("HIDDEN_LAYERS"))
dropout = float(os.getenv("DROPOUT"))
num_epochs = int(os.getenv("NUM_EPOCHS"))

In [ ]:
train_dataloader, validation_dataloader, test_dataloader = define_dataloader_from_angle_dataset(train_data, test_data, batch_size, split_size=0.95)

## Load and Train the model

In [ ]:
from models.fully_connected import FullyConnected, train
from utils.loss_functions import get_loss_function
from utils.optimizer import get_optimizer_function
from utils.activation import get_activation
from utils.early_stopping import EarlyStopping

In [ ]:
early_stopping = EarlyStopping(10)

In [ ]:
activation = get_activation(hidden_activation)
model = FullyConnected(input_shape, hidden_layers, output_shape, dropout, activation)
optimizer = get_optimizer_function(optimizer, model, learning_rate)
loss_function = get_loss_function()

In [ ]:
_, validation_losses = train(num_epochs, train_dataloader, validation_dataloader, model, loss_function, optimizer, model_path, device, early_stopping=early_stopping)

In [ ]:
model.eval()

## Evaluation

In [ ]:
from utils.evaluation import compute_losses_from, compute_predictions
from utils.visualization import create_plot_for_dimensions
import matplotlib.pyplot as plt

from typing import Sequence

In [ ]:
model_state_dict = torch.load(model_path / "checkpoint.pt")
model.load_state_dict(model_state_dict)

## 1. Analysis: Compute test loss

In [ ]:
y, y_true, inference_times = compute_predictions(test_dataloader, model, 'cpu')
test_losses = compute_losses_from(y, y_true, loss_function)
print(f"The mean squared error of the loaded model on test is: {test_losses.mean()}")

## 2. Analysis: Show Cable lowest points

In [ ]:
plot = create_plot_for_dimensions(y.numpy(), y_true.numpy(), size=5)
plot.savefig(model_path / "predictions.png")
plot.show()

In [ ]:
def create_inference_time_plot(inference_times: np.ndarray, figsize: Sequence[float] = None):
    fig = plt.figure(figsize=figsize)
    plt.plot(inference_times)
    plt.title("Inference time per input in visualization dataset")
    plt.ylabel("Time in s")
    plt.xticks([])
    return plt

In [ ]:
plot = create_inference_time_plot(inference_times.numpy())
plot.savefig(model_path / "inference_times.png")
plot.show()

## 4. Analysis: Show validation loss over time

In [ ]:
def create_validation_loss_plot(validation_losses: np.ndarray, figsize: Sequence[float] = None):
    fig = plt.figure(figsize=figsize)
    plt.plot(validation_losses)
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.title("Validation loss over time")
    return plt

In [ ]:
plot = create_validation_loss_plot(validation_losses)
plot.savefig(model_path / "losses.png")
plot.show()